In [37]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import RandomForestRegressor as rfr

In [49]:
data = pd.read_csv("./data/train/01.csv")
# 確実にいらないカラムたち
data = data.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
# もしかしたら使うかもしれないカラムたち
data = data.drop(['市区町村コード','都市計画'],axis=1)
data = data.fillna({'改装':'未改装','容積率（％）':0,'建ぺい率（％）':0})
data = data.drop(['間取り','建築年','建物の構造','取引時点','最寄駅：距離（分）'],axis = 1)
dummies = pd.get_dummies(data[['改装']])
data = pd.concat([data.drop('改装',axis=1),dummies[['改装_改装済']]],axis = 1)
data

,面積（㎡）,建ぺい率（％）,容積率（％）,取引価格（総額）_log,改装_改装済
0,85,80.0,400.0,6.963788,1
1,70,60.0,200.0,7.000000,0
2,70,60.0,200.0,7.301030,1
3,85,60.0,300.0,7.255273,0
4,20,80.0,600.0,6.361728,0
...,...,...,...,...,...
24357,90,60.0,200.0,7.113943,1
24358,60,80.0,300.0,7.079181,1
24359,100,80.0,300.0,7.397940,0
24360,95,80.0,300.0,7.431364,0


In [48]:
test = pd.read_csv('./data/test.csv')
test = test.drop(['ID','種類','地域', '都道府県名', '市区町村名', '地区名','土地の形状', '間口', 
                                  '延床面積（㎡）','用途','今後の利用目的','前面道路：方位', '前面道路：種類', 
                                  '前面道路：幅員（ｍ）','取引の事情等','最寄駅：名称'],axis=1) 
# もしかしたら使うかもしれないカラムたち
test = test.drop(['市区町村コード','都市計画'],axis=1)
test = test.fillna({'改装':'未改装','容積率（％）':0,'建ぺい率（％）':0})
test = test.drop(['間取り','建築年','建物の構造','取引時点','最寄駅：距離（分）'],axis = 1)
dummies = pd.get_dummies(test[['改装']])
test_X = pd.concat([test.drop('改装',axis=1),dummies[['改装_改装済']]],axis = 1)
test_X

,面積（㎡）,建ぺい率（％）,容積率（％）,改装_改装済
0,75,80.0,600.0,0
1,30,80.0,400.0,1
2,70,80.0,400.0,1
3,50,80.0,400.0,0
4,45,80.0,400.0,1
...,...,...,...,...
21000,80,60.0,200.0,0
21001,55,60.0,200.0,0
21002,65,70.0,200.0,0
21003,80,80.0,400.0,0


In [50]:
train_X = data.drop('取引価格（総額）_log',axis = 1)
train_y = data['取引価格（総額）_log']

In [51]:
forest = rfr(random_state = 1010)
forest.fit(train_X,train_y)
pred = forest.predict(test_X)

In [45]:
test_X.isna().sum()

面積（㎡）        0
建ぺい率（％）    409
容積率（％）     409
改装_改装済       0
dtype: int64

In [46]:
test_X.isna().sum()

面積（㎡）        0
建ぺい率（％）    409
容積率（％）     409
改装_改装済       0
dtype: int64

In [78]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['取引価格（総額）_log']=pred
submit.to_csv('./submits/first_submit.csv',index=False)

In [77]:
submit

,ID,取引価格（総額）_log
0,1000078,7.391586
1,1000121,6.558229
2,1000123,7.118635
3,1000127,6.817129
4,1000129,6.881544
...,...,...
21000,47003951,7.116549
21001,47006115,6.673206
21002,47006712,6.861716
21003,47023667,7.279903
